In [14]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from collections import Counter
from tensorflow.keras import layers
from tensorflow.keras.backend import sparse_categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten
from transformers import BertTokenizer, TFBertModel

In [2]:
!ls

corpus-webis-tldr-17.json  DataProcessing.ipynb
corpus-webis-tldr-17.zip   README.md


In [3]:
!head -1 corpus-webis-tldr-17.json | jq 'keys'

[
  "author",
  "body",
  "content",
  "content_len",
  "id",
  "normalizedBody",
  "subreddit",
  "subreddit_id",
  "summary",
  "summary_len"
]


In [4]:
!head -1 corpus-webis-tldr-17.json

{"author":"raysofdarkmatter","body":"I think it should be fixed on either UTC standard or UTC+1 year around, with the current zone offsets.\n\nMoving timescales add a lot of complexity to the implementation of timekeeping systems and have [dubious value]( \n\nI think seasonal shifting time made sense in the pre-electric past, when timekeeping was more flexible and artificial light was inefficient and often dangerous. \n\nNow we have machines that work easily with simple timekeeping rules, and it's more beneficial to spend a small amount on energy for lighting, and save the larger cost of engineering things to work with the complex timekeeping rules, as well as saving the irritation to humans.\n\nLighting has gotten much more efficient over time; we can squeeze out a lot more photons per unit of energy from a 2012 CFL or LED than a candle could in 1780, or a lightbulb could in 1950. \n\nThere's a lot of room for improvement in how we use lights as well; as lighting control gets more int

In [25]:
reddit_posts = []
with open('corpus-webis-tldr-17.json', 'r') as f:
    for i, line in enumerate(f):
        post = json.loads(line)
        del post['body']
        del post['normalizedBody']
        reddit_posts.append(post)
        if i % 100000 == 0:
            print(i)

In [26]:
reddit_posts[3:6]

[{'author': 'NightlyReaper',
  'content': 'In Mechwarrior Online, I have begun to use a mouse for the "turret" aspect of the torso twist/tilt and a Logitech G13 for other controls (  The G-13 has a little thumbstick that works well for WASD functions and has a little keyboard to cover all the other (limited) controls required for MWO.  For modern games, I guess that\'s as close to HOTAS as they want you to get.  For a better solution, I think that with a joystick emulating mouse inputs it would be passable, but as you said:  It\'s presently a sad day and you have to be able to edit XML files to modify joystick input triggers and it starts becoming an occupation rather than a gaming hobby.  In Mechwarrior 2, 3, & 4 I use the Steel Battalion controller and it is freaking awesome to have the three axes of torso twist, tilt, walking direction, plus a real throttle, plus real hat switches for views, plus pedals for extra speed, stopping power and jumpjet control.  Not to mention the 32 othe

In [29]:
# [p for p in reddit_posts if p['subreddit'] == 'Borderlands']
subreddits = Counter()
missing = 0
for p in reddit_posts:
    if 'subreddit' not in p:
        missing+=1
    else:
        subreddits[p['subreddit']] += 1

In [30]:
missing

136

In [31]:
subreddits.most_common(100)


[('AskReddit', 589947),
 ('relationships', 352049),
 ('leagueoflegends', 109307),
 ('tifu', 52219),
 ('relationship_advice', 50416),
 ('trees', 47286),
 ('gaming', 43851),
 ('atheism', 43268),
 ('AdviceAnimals', 40783),
 ('funny', 40171),
 ('politics', 36518),
 ('pics', 35098),
 ('sex', 28806),
 ('WTF', 25781),
 ('explainlikeimfive', 25482),
 ('todayilearned', 25004),
 ('Fitness', 22694),
 ('IAmA', 22689),
 ('worldnews', 22577),
 ('DotA2', 22405),
 ('TwoXChromosomes', 21562),
 ('videos', 20567),
 ('DestinyTheGame', 19878),
 ('reddit.com', 19020),
 ('offmychest', 17175),
 ('buildapc', 14817),
 ('AskMen', 14538),
 ('personalfinance', 14403),
 ('summonerschool', 13806),
 ('technology', 13700),
 ('wow', 13309),
 ('NoFap', 13203),
 ('starcraft', 12911),
 ('dating_advice', 12361),
 ('askscience', 12037),
 ('Games', 11820),
 ('news', 11660),
 ('talesfromtechsupport', 11537),
 ('depression', 11365),
 ('pcmasterrace', 11348),
 ('Guildwars2', 10948),
 ('magicTCG', 10624),
 ('loseit', 10559),
 ('

In [23]:
subreddits.least_common(10)


AttributeError: 'Counter' object has no attribute 'least_common'